Cell 1: Setup the environment

Set up the environment by loading environment variables, suppressing warnings, and preparing for model deprecation management.

In [ ]:
# Import necessary modules
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv())  # Load .env file

# Suppress warnings related to deprecated functions or modules
import warnings
warnings.filterwarnings('ignore')


Cell 2: Handle LLM model deprecation based on the current date

Select the correct LLM model based on the current date, switching to a newer model after a specified target date.



In [ ]:
# Import datetime module to handle date-based model selection
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the cutoff date after which the newer model is used
target_date = datetime.date(2024, 6, 12)

# Conditionally set the LLM model based on whether the current date is before or after the target date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


Create a conversation chain using LangChain's ConversationBufferMemory, which stores the full conversation history for future use.



In [ ]:
# Import necessary classes from LangChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Initialize the LLM with the selected model and zero temperature (deterministic output)
llm = ChatOpenAI(temperature=0.0, model=llm_model)

# Initialize ConversationBufferMemory to store the entire conversation history
memory = ConversationBufferMemory()

# Create a conversation chain that uses the initialized LLM and memory
conversation = ConversationChain(
    llm=llm, 
    memory=memory, 
    verbose=True  # Enable detailed output of conversation processing
)

# Simulate a conversation with three turns
conversation.predict(input="Hi, my name is Andrew")
conversation.predict(input="What is 1+1?")
conversation.predict(input="What is my name?")


Cell 4: Inspect and manipulate ConversationBufferMemory

Demonstrate how to inspect and manipulate the conversation buffer by saving context and loading variables.



In [ ]:
# Print the buffer which contains the full conversation history
print(memory.buffer)

# Load memory variables, this can be used to retrieve saved conversation context
memory.load_memory_variables({})

# Reset memory and manually save a new conversation context
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, {"output": "What's up"})
print(memory.buffer)  # Print the buffer after saving new context

# Save another conversation context to the memory and inspect the buffer again
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
print(memory.buffer)  # Print the updated buffer


Cell 5: Using ConversationBufferWindowMemory to store only recent conversations


Use ConversationBufferWindowMemory to store only the most recent k number of conversation exchanges, rather than the entire conversation history.

In [ ]:
# Import ConversationBufferWindowMemory class
from langchain.memory import ConversationBufferWindowMemory

# Initialize ConversationBufferWindowMemory to store only the last k=1 exchanges
memory = ConversationBufferWindowMemory(k=1)

# Save a few conversation exchanges into the windowed memory
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

# Load memory variables to inspect what is stored (only stores the last `k=1` exchanges)
print(memory.load_memory_variables({}))

# Create a new conversation chain using this windowed memory
llm = ChatOpenAI(temperature=0.0, model=llm_model)
conversation = ConversationChain(
    llm=llm, 
    memory=memory, 
    verbose=False  # Disable verbose to suppress intermediate outputs
)

# Simulate a conversation using the windowed memory
conversation.predict(input="Hi, my name is Andrew")
conversation.predict(input="What is 1+1?")
conversation.predict(input="What is my name?")


Cell 6: Using ConversationTokenBufferMemory to limit token usage

Use ConversationTokenBufferMemory to limit the number of tokens stored in memory, ensuring that only recent context is kept within the token limit.



In [ ]:
# Import ConversationTokenBufferMemory class from LangChain
from langchain.memory import ConversationTokenBufferMemory

# Initialize token-buffered memory with a maximum token limit
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

# Save several exchanges into memory, each contributing to the token count
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

# Load and inspect the memory, it should store only exchanges that fit within the 50-token limit
print(memory.load_memory_variables({}))


Cell 7: Using ConversationSummaryBufferMemory to summarize conversations

Use ConversationSummaryBufferMemory to summarize long conversations, retaining key points while limiting the token count to a manageable level.



In [ ]:
# Import ConversationSummaryBufferMemory class
from langchain.memory import ConversationSummaryBufferMemory

# Create a long string that simulates a schedule for demonstration
schedule = ("There is a meeting at 8am with your product team. "
            "You will need your powerpoint presentation prepared. "
            "9am-12pm have time to work on your LangChain project which will go quickly "
            "because Langchain is such a powerful tool. At Noon, lunch at the italian restaurant "
            "with a customer who is driving from over an hour away to meet you to understand the latest in AI. "
            "Be sure to bring your laptop to show the latest LLM demo.")

# Initialize summary buffer memory with a token limit
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

# Save several exchanges into the memory, including the detailed schedule
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, {"output": f"{schedule}"})

# Load and inspect the summarized memory variables (summary of the conversation)
print(memory.load_memory_variables({}))

# Create a conversation chain using the summarized memory
conversation = ConversationChain(
    llm=llm, 
    memory=memory, 
    verbose=True  # Enable verbose to show the summary in action
)

# Make a prediction to check how the memory summarization works
conversation.predict(input="What would be a good demo to show?")


Suggestions for further improvements:
a. Add unit tests to validate each type of memory behavior.
b. Implement persistence to save and load memory from files to maintain conversation history across sessions.





